<a href="https://colab.research.google.com/github/peculab/AI4JUBO/blob/main/JuboDeath_V9_pureData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 訓練資料是 mortality_2020_2023_1014/training_data_1014
#### 外部驗證資料是 mortality_2024_1014/external_validation_1014

#### 次族群

- <= 85 & > 85
- ADL 變好 & ADL 變差
- 男性 & 女性

#### 由於各項量測數值有限制在６個月內的量測值，且有新增體重的變化，因此 ADL 沒有值被排除的人比較多。

In [1]:
!pip install shap plotly xgboost --quiet

In [2]:
!pip uninstall shap -y
!pip install shap --no-deps

Found existing installation: shap 0.48.0
Uninstalling shap-0.48.0:
  Successfully uninstalled shap-0.48.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.9 MB/s eta 0:00:00


In [3]:
!pip install ace_tools

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
from IPython.display import display
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    classification_report, confusion_matrix, mean_absolute_error, r2_score
)

In [6]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

外部資料讀入

In [7]:
# read data and put it in a dataframe
# 在 google 工作表載入外部資料 gsheets

gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1NFAhP8NUVsxzEq55siFA0yHvnXY5GWqiKGSOKC4y1Qg/edit?usp=sharing')
worksheet = gsheets.worksheet("external_validation_1014")  # 指定分頁名稱

worksheet = worksheet.get_all_records()
external = pd.DataFrame(worksheet)
external = external.apply(lambda col: pd.to_numeric(col.astype(str).str.replace(',', '').str.strip(), errors='coerce'))
external.head()

,H01_NUM,dbname,入家日期,結案日期,死亡標記,觀察天數,性別_is_male,預估年齡,DNR_flag,ADL_總分_max,...,意識總分_diff,意識分級,使用呼吸輔具,first_has_feeding_tube,last_has_feeding_tube,diff_has_feeding_tube,had_fall,BW_first,BW_last,BW_diff_seq
0,1376,NaN,NaN,NaN,0,197,1,77,0,90,...,0.0,NaN,0.0,0.0,0.0,0.0,0.0,74.0,74.1,0.1
1,1322,NaN,NaN,NaN,0,327,1,92,0,10,...,0.0,NaN,0.0,0.0,0.0,0.0,0.0,75.3,69.2,-6.1
2,1319,NaN,NaN,NaN,0,255,1,78,1,5,...,-1.0,NaN,0.0,0.0,0.0,0.0,0.0,46.0,35.7,-10.3
3,1333,NaN,NaN,NaN,0,293,1,82,1,10,...,0.0,NaN,0.0,0.0,0.0,0.0,0.0,58.7,53.4,-5.3
4,1452,NaN,NaN,NaN,0,341,0,80,1,10,...,-1.0,NaN,0.0,0.0,0.0,0.0,0.0,40.7,36.5,-4.2


In [8]:
external.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
H01_NUM,6216.0,1391.613256,770.503700,4.0,1187.000,1267.000,1388.250000,20295.000000
dbname,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
入家日期,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
結案日期,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
死亡標記,6216.0,0.286519,0.452171,0.0,0.000,0.000,1.000000,1.000000
觀察天數,6216.0,208.509492,110.248977,0.0,121.000,232.000,297.000000,365.000000
性別_is_male,6216.0,0.510457,0.499931,0.0,0.000,1.000,1.000000,1.000000
預估年齡,6216.0,78.604086,11.724613,1.0,72.000,81.000,87.000000,124.000000
DNR_flag,6216.0,0.455598,0.498065,0.0,0.000,0.000,1.000000,1.000000
ADL_總分_max,6216.0,28.906853,31.175410,0.0,0.000,20.000,50.000000,100.000000


In [9]:
ex_missing_info = external.isnull().sum().to_frame(name='Missing Count')
ex_missing_info['Missing Ratio'] = (ex_missing_info['Missing Count'] / len(external)).round(4)
ex_missing_info = ex_missing_info.sort_values(by='Missing Ratio', ascending=True)
ex_missing_info

,Missing Count,Missing Ratio
H01_NUM,0,0.0000
性別_is_male,0,0.0000
觀察天數,0,0.0000
死亡標記,0,0.0000
預估年齡,0,0.0000
DNR_flag,0,0.0000
ADL_總分_max,0,0.0000
ADL_明顯惡化,0,0.0000
六個月內住院次數,0,0.0000
ADL_last_CouldNot,0,0.0000


訓練資料讀入

In [10]:
# read data and put it in a dataframe
# 在 google 工作表載入訓練資料 gsheets

gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1qljyp9lq3QsZ7O2O7FQxm7taEWQi3F3bZgNMcQ7NJeE/edit?usp=sharing')
worksheet = gsheets.worksheet("training_data_1014")  # 指定分頁名稱

worksheet = worksheet.get_all_records()
df = pd.DataFrame(worksheet)
df = df.apply(lambda col: pd.to_numeric(col.astype(str).str.replace(',', '').str.strip(), errors='coerce'))
df.head()

,H01_NUM,dbname,入家日期,結案日期,死亡標記,觀察天數,性別_is_male,預估年齡,DNR_flag,ADL_總分_max,...,意識總分_diff,意識分級,使用呼吸輔具,first_has_feeding_tube,last_has_feeding_tube,diff_has_feeding_tube,had_fall,BW_first,BW_last,BW_diff_seq
0,1325,NaN,NaN,NaN,0,739,1,66,1,0,...,0.0,NaN,0.0,0.0,0.0,0.0,0.0,55.50,51.00,-4.50
1,1160,NaN,NaN,NaN,0,788,1,89,1,95,...,0.0,NaN,0.0,0.0,0.0,0.0,0.0,50.25,46.70,-3.55
2,1253,NaN,NaN,NaN,0,1292,0,89,0,60,...,0.0,NaN,0.0,0.0,0.0,0.0,0.0,63.10,64.05,0.95
3,1342,NaN,NaN,NaN,0,584,0,93,1,30,...,0.0,NaN,0.0,0.0,0.0,0.0,0.0,55.50,52.65,-2.85
4,1343,NaN,NaN,NaN,0,583,1,90,0,90,...,0.0,NaN,0.0,0.0,0.0,0.0,0.0,66.85,65.60,-1.25


In [11]:
df.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
H01_NUM,23901.0,1457.124723,3207.707525,1.0,1174.0,1233.0,1327.000000,100463.000000
dbname,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
入家日期,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
結案日期,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
死亡標記,23901.0,0.220577,0.414644,0.0,0.0,0.0,0.000000,1.000000
觀察天數,23901.0,584.830007,428.471637,0.0,211.0,509.0,957.000000,1460.000000
性別_is_male,23901.0,0.491444,0.499937,0.0,0.0,0.0,1.000000,1.000000
預估年齡,23901.0,79.317476,11.842274,0.0,72.0,82.0,88.000000,125.000000
DNR_flag,23901.0,0.409062,0.491671,0.0,0.0,0.0,1.000000,1.000000
ADL_總分_max,23901.0,31.028200,32.745984,0.0,0.0,20.0,55.000000,100.000000


In [12]:
df_missing_info = df.isnull().sum().to_frame(name='Missing Count')
df_missing_info['Missing Ratio'] = (df_missing_info['Missing Count'] / len(df)).round(4)
df_missing_info = df_missing_info.sort_values(by='Missing Ratio', ascending=True)
df_missing_info

,Missing Count,Missing Ratio
H01_NUM,0,0.0000
性別_is_male,0,0.0000
觀察天數,0,0.0000
死亡標記,0,0.0000
預估年齡,0,0.0000
DNR_flag,0,0.0000
ADL_總分_max,0,0.0000
ADL_明顯惡化,0,0.0000
六個月內住院次數,0,0.0000
ADL_last_CouldNot,0,0.0000


In [13]:
features = df_missing_info[df_missing_info['Missing Ratio']<0.3].index.tolist()

In [14]:
features

['H01_NUM',
 '性別_is_male',
 '觀察天數',
 '死亡標記',
 '預估年齡',
 'DNR_flag',
 'ADL_總分_max',
 'ADL_明顯惡化',
 '六個月內住院次數',
 'ADL_last_CouldNot',
 'ADL_first_CouldNot',
 'ADL_first_score',
 'ADL_Max',
 'ADL_Min',
 'last_has_denture',
 'diff_has_denture',
 'first_has_denture',
 'first_has_feeding_tube',
 'last_has_feeding_tube',
 'diff_has_feeding_tube',
 'last_ 意識總分',
 '意識總分Max',
 '意識總分_diff',
 'had_fall',
 '使用呼吸輔具',
 'first_ 意識總分',
 'BW_first',
 'BW_last',
 'BW_diff_seq',
 'ADL_std',
 'ADL_last_score',
 'ADL_diff_seq']

In [15]:
dfNew = df[features]

In [16]:
dfNew = dfNew.fillna(0)

In [17]:
from sklearn.base import BaseEstimator, ClassifierMixin

class HybridXGBRF(BaseEstimator, ClassifierMixin):
    def __init__(self, xgb_model=None, rf_model=None, alpha=0.5):
        self.xgb_model = xgb_model
        self.rf_model = rf_model
        self.alpha = alpha
        self._init_models()

    def _init_models(self):
        # Best Parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 800, 'subsample': 1.0}
        # "XGBClassifier": XGBClassifier(n_estimators=200, learning_rate=0.01, max_depth=5, random_state=42, eval_metric='logloss'),

        self.xgb = self.xgb_model or XGBClassifier(
            eval_metric="logloss",
            random_state=42,
            colsample_bytree=0.8,     # ✅ 降低每棵樹看到的特徵比例 → 提高多樣性
            learning_rate=0.01,       # ✅ 稍微提升學習率搭配更早停止
            max_depth=5,              # ✅ 降低單棵樹複雜度 → 降低過擬合
            n_estimators=200,         # ✅ 總樹數可略減以免累積錯誤
            subsample=1.0,            # ✅ 樣本隨機抽樣 → 提升隨機性
            verbosity=0,
            use_label_encoder=False
        )
        self.rf = self.rf_model or RandomForestClassifier(
            n_estimators=100,
            random_state=42
        )

    def fit(self, X, y):
        self._init_models()  # 每次 fit 要重設模型
        self.xgb.fit(X, y)
        self.rf.fit(X, y)
        return self

    def predict_proba(self, X):
        xgb_prob = self.xgb.predict_proba(X)[:, 1]
        rf_prob = self.rf.predict_proba(X)[:, 1]
        blended = self.alpha * xgb_prob + (1 - self.alpha) * rf_prob
        return np.vstack([1 - blended, blended]).T

    def predict(self, X):
        return (self.predict_proba(X)[:, 1] > 0.5).astype(int)

    def get_params(self, deep=True):
        return {
            'xgb_model': self.xgb_model,
            'rf_model': self.rf_model,
            'alpha': self.alpha
        }

    def set_params(self, **params):
        for param, value in params.items():
            setattr(self, param, value)
        self._init_models()  # 重新初始化模型
        return self

In [18]:
# 要移除的欄位，是代表身分標記，以及天數
drop_columns = ['H01_NUM', '觀察天數']

# 丟掉這些欄位
dfNew = dfNew.drop(columns=drop_columns)

In [19]:
dfNew

,性別_is_male,死亡標記,預估年齡,DNR_flag,ADL_總分_max,ADL_明顯惡化,六個月內住院次數,ADL_last_CouldNot,ADL_first_CouldNot,ADL_first_score,...,意識總分_diff,had_fall,使用呼吸輔具,first_ 意識總分,BW_first,BW_last,BW_diff_seq,ADL_std,ADL_last_score,ADL_diff_seq
0,1,0,66,1,0,0,0,0,0,0.0,...,0.0,0.0,0.0,3.0,55.50,51.00,-4.50,0.000000,0.0,0.0
1,1,0,89,1,95,1,0,0,0,95.0,...,0.0,0.0,0.0,13.0,50.25,46.70,-3.55,49.074773,10.0,-85.0
2,0,0,89,0,60,0,0,0,0,60.0,...,0.0,0.0,0.0,15.0,63.10,64.05,0.95,0.000000,60.0,0.0
3,0,0,93,1,30,0,0,0,0,30.0,...,0.0,0.0,0.0,14.0,55.50,52.65,-2.85,0.000000,30.0,0.0
4,1,0,90,0,90,0,0,0,0,90.0,...,0.0,0.0,0.0,15.0,66.85,65.60,-1.25,0.000000,90.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23896,1,0,48,0,100,0,0,0,0,100.0,...,0.0,0.0,0.0,3.0,69.80,70.00,0.20,0.000000,100.0,0.0
23897,1,0,41,0,100,0,0,0,0,100.0,...,0.0,0.0,0.0,3.0,70.00,68.30,-1.70,0.000000,0.0,0.0
23898,0,0,60,0,100,0,0,0,0,100.0,...,0.0,0.0,0.0,3.0,51.90,47.30,-4.60,0.000000,100.0,0.0
23899,0,0,54,0,100,0,0,0,0,100.0,...,0.0,0.0,0.0,3.0,50.00,53.30,3.30,0.000000,100.0,0.0


In [20]:
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import (
    confusion_matrix, roc_curve, auc, classification_report
)
import plotly.graph_objects as go
import plotly.express as px

In [21]:
# === 資料準備 ===
X = dfNew.drop(columns=['死亡標記'])
y = df['死亡標記']

In [22]:
X_missing_info = X.isnull().sum().to_frame(name='Missing Count')
X_missing_info['Missing Ratio'] = (X_missing_info['Missing Count'] / len(X)).round(4)
X_missing_info = X_missing_info.sort_values(by='Missing Ratio', ascending=True)
X_missing_info

,Missing Count,Missing Ratio
性別_is_male,0,0.0
預估年齡,0,0.0
DNR_flag,0,0.0
ADL_總分_max,0,0.0
ADL_明顯惡化,0,0.0
六個月內住院次數,0,0.0
ADL_last_CouldNot,0,0.0
ADL_first_CouldNot,0,0.0
ADL_first_score,0,0.0
ADL_Max,0,0.0


# 開始進行訓練

In [23]:
X.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
性別_is_male,23901.0,0.491444,0.499937,0.0,0.00,0.0,1.000000,1.000000
預估年齡,23901.0,79.317476,11.842274,0.0,72.00,82.0,88.000000,125.000000
DNR_flag,23901.0,0.409062,0.491671,0.0,0.00,0.0,1.000000,1.000000
ADL_總分_max,23901.0,31.028200,32.745984,0.0,0.00,20.0,55.000000,100.000000
ADL_明顯惡化,23901.0,0.111962,0.315326,0.0,0.00,0.0,0.000000,1.000000
六個月內住院次數,23901.0,0.650056,1.055352,0.0,0.00,0.0,1.000000,12.000000
ADL_last_CouldNot,23901.0,0.018242,0.133828,0.0,0.00,0.0,0.000000,1.000000
ADL_first_CouldNot,23901.0,0.017238,0.130159,0.0,0.00,0.0,0.000000,1.000000
ADL_first_score,23901.0,28.220367,31.699312,0.0,0.00,15.0,50.000000,100.000000
ADL_Max,23901.0,31.028200,32.745984,0.0,0.00,20.0,55.000000,100.000000


In [24]:
!pip install lifelines

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.3/117.3 kB 12.0 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4030 sha256=3259b09ce594051edfd7688c5fdb34919d5bb379f0f21806c87781fbfd403ee4
  Stored in directory: /root/.cache/pip/wheels/50/37/21/0a719b9d89c635e89ff24bd93b862882ad675279552013b2fb
Successfully built autograd-gamma


In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lifelines import CoxPHFitter
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegressionCV

# 假設 HybridXGBRF 已定義
all_models = {
    #"HybridXGBRF (Our Approach)": HybridXGBRF(alpha=1),
    "HybridXGBRF (Our Approach)": XGBClassifier(n_estimators=200, learning_rate=0.01, max_depth=5, random_state=42, eval_metric='logloss', subsample=1.0, verbosity=0),
    "LogisticRegression (max_iter=200)": LogisticRegression(max_iter=200),
    "XGBClassifier": XGBClassifier(n_estimators=500, learning_rate=0.01, max_depth=3, random_state=42, eval_metric='logloss'),
    "RandomForestClassifier": RandomForestClassifier(n_estimators=100, random_state=42),
    "LogisticRegression (max_iter=1000)": LogisticRegression(max_iter=1000),

    # 🔽 新增未測試模型
    "Ridge": make_pipeline(StandardScaler(), LogisticRegression(penalty='l2', solver='saga', max_iter=1000, random_state=42)),
    "Lasso": make_pipeline(StandardScaler(), LogisticRegression(penalty='l1', solver='saga', max_iter=1000, random_state=42)),
    "Elastic": make_pipeline(StandardScaler(), LogisticRegression(penalty='elasticnet', solver='saga', l1_ratio=0.5, max_iter=1000, random_state=42)),
}

In [26]:
import copy
from sklearn.base import clone
from sklearn.impute import SimpleImputer
from sklearn.exceptions import NotFittedError
from sklearn.utils.validation import check_is_fitted

# Prepare CV and ROC Figure
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
fig_roc = go.Figure()
mean_fpr = np.linspace(0, 1, 100)

results = []
trained_models = {}

for model_name, model in all_models.items():
    print(f"▶ Running CV for: {model_name}")
    accs, precs, recalls, f1s, aucs = [], [], [], [], []
    all_cm = np.zeros((2, 2), dtype=int)
    tprs = []

    for fold, (train_idx, test_idx) in enumerate(skf.split(X, y)):
        X_train, X_test = X.iloc[train_idx].copy(), X.iloc[test_idx].copy()
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        model_fold = clone(model)

        try:
            # 嘗試直接 fit（若模型支援 NaN，會成功）
            model_fold.fit(X_train, y_train)

        except ValueError as e:
            if "Input X contains NaN" in str(e):
                print(f"⚠️ Missing value detected for {model_name} (fold {fold+1}) — applying median imputation.")

                imputer = SimpleImputer(strategy='median')
                X_train = pd.DataFrame(imputer.fit_transform(X_train), columns=X.columns)
                X_test = pd.DataFrame(imputer.transform(X_test), columns=X.columns)

                model_fold.fit(X_train, y_train)
            else:
                raise e  # 若是其他錯誤就直接拋出

        y_pred = model_fold.predict(X_test)
        y_prob = model_fold.predict_proba(X_test)[:, 1]

        # ROC
        fpr, tpr, _ = roc_curve(y_test, y_prob)
        roc_auc = auc(fpr, tpr)
        tpr_interp = np.interp(mean_fpr, fpr, tpr)
        tpr_interp[0] = 0.0
        tprs.append(tpr_interp)
        aucs.append(roc_auc)

        # Metrics
        accs.append(accuracy_score(y_test, y_pred))
        precs.append(precision_score(y_test, y_pred))
        recalls.append(recall_score(y_test, y_pred))
        f1s.append(f1_score(y_test, y_pred))
        all_cm += confusion_matrix(y_test, y_pred)

    trained_models[model_name] = copy.deepcopy(model_fold)

    # ROC Curve
    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)

    fig_roc.add_trace(go.Scatter(
        x=mean_fpr, y=mean_tpr, mode='lines',
        name=f"{model_name} (mean AUC={mean_auc:.3f})"
    ))

    results.append({
        'Model': model_name,
        'Accuracy Mean': np.mean(accs),
        'Accuracy Std': np.std(accs),
        'Precision Mean': np.mean(precs),
        'Recall Mean': np.mean(recalls),
        'F1 Score Mean': np.mean(f1s),
        'ROC AUC Mean': np.mean(aucs),
        'ROC AUC Std': np.std(aucs),
        'Confusion Matrix': all_cm
    })

# Add Random Baseline
fig_roc.add_trace(go.Scatter(
    x=[0, 1], y=[0, 1], mode='lines',
    line=dict(dash='dash'), name='Random Baseline'
))

fig_roc.update_layout(
    title="ROC Curve Comparison (Cross-Validation Mean)",
    xaxis_title="False Positive Rate",
    yaxis_title="True Positive Rate",
    width=800, height=600
)

fig_roc.show()

▶ Running CV for: HybridXGBRF (Our Approach)
▶ Running CV for: LogisticRegression (max_iter=200)


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

/usr/local/lib/python3.12/dist-packages/sklearn/

▶ Running CV for: XGBClassifier
▶ Running CV for: RandomForestClassifier
▶ Running CV for: LogisticRegression (max_iter=1000)


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

/usr/local/lib/python3.12/dist-packages/sklearn/

▶ Running CV for: Ridge


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge



▶ Running CV for: Lasso


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge



▶ Running CV for: Elastic


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge



In [27]:
import shap

# 訓練模型（完整資料）
xgb_model = all_models["HybridXGBRF (Our Approach)"]
xgb_model.fit(X, y)

# 建立 SHAP 解釋器
explainer = shap.TreeExplainer(xgb_model)
shap_values = explainer.shap_values(X)

# 計算平均 SHAP 值絕對值（作為重要性）
import numpy as np
shap_abs_mean = np.abs(shap_values).mean(axis=0)

In [28]:
import pandas as pd
import plotly.express as px

# 整理成 DataFrame 並排序
importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Mean |SHAP Value|': shap_abs_mean
}).sort_values(by='Mean |SHAP Value|', ascending=False)

# 畫前 20 名
top_n = 20
fig_bar = px.bar(
    importance_df.head(top_n),
    x='Mean |SHAP Value|',
    y='Feature',
    orientation='h',
    title="🎯 Top SHAP Features by Mean |SHAP|",
)
fig_bar.update_layout(yaxis=dict(categoryorder='total ascending'))
fig_bar.show()

In [29]:
import plotly.graph_objects as go

for r in results:
    cm = r['Confusion Matrix']
    model_name = r['Model']

    z = cm
    x_labels = ['Predicted 0', 'Predicted 1']
    y_labels = ['Actual 0', 'Actual 1']

    fig_cm = go.Figure(data=go.Heatmap(
        z=z,
        x=x_labels,
        y=y_labels,
        colorscale='Blues',
        text=z,
        texttemplate="%{text}"
    ))

    fig_cm.update_layout(
        title=f"Confusion Matrix - {model_name}",
        xaxis_title="Predicted Label",
        yaxis_title="True Label",
        width=500,
        height=500
    )

    fig_cm.show()

In [30]:
# Convert results to DataFrame
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by="ROC AUC Mean", ascending=False).reset_index(drop=True)
results_df

,Model,Accuracy Mean,Accuracy Std,Precision Mean,Recall Mean,F1 Score Mean,ROC AUC Mean,ROC AUC Std,Confusion Matrix
0,XGBClassifier,0.849923,0.004088,0.774242,0.451443,0.570269,0.875269,0.006163,"[[17934, 695], [2892, 2380]]"
1,HybridXGBRF (Our Approach),0.853437,0.003076,0.789438,0.457703,0.579377,0.874455,0.006134,"[[17985, 644], [2859, 2413]]"
2,RandomForestClassifier,0.846366,0.004392,0.724948,0.489377,0.584221,0.857591,0.008462,"[[17649, 980], [2692, 2580]]"
3,LogisticRegression (max_iter=1000),0.792101,0.004096,0.581470,0.205238,0.303368,0.796807,0.007380,"[[17850, 779], [4190, 1082]]"
4,Ridge,0.792143,0.004107,0.583156,0.202583,0.300663,0.795719,0.008014,"[[17865, 764], [4204, 1068]]"
5,Elastic,0.792143,0.003988,0.583194,0.202583,0.300668,0.795709,0.008039,"[[17865, 764], [4204, 1068]]"
6,Lasso,0.792101,0.003894,0.583143,0.202014,0.300031,0.795697,0.008068,"[[17867, 762], [4207, 1065]]"
7,LogisticRegression (max_iter=200),0.792143,0.003182,0.579225,0.211117,0.309308,0.795177,0.007978,"[[17820, 809], [4159, 1113]]"


# 測試外部資料在 XGBoost 模型下的結果

In [31]:
external.head()

,H01_NUM,dbname,入家日期,結案日期,死亡標記,觀察天數,性別_is_male,預估年齡,DNR_flag,ADL_總分_max,...,意識總分_diff,意識分級,使用呼吸輔具,first_has_feeding_tube,last_has_feeding_tube,diff_has_feeding_tube,had_fall,BW_first,BW_last,BW_diff_seq
0,1376,NaN,NaN,NaN,0,197,1,77,0,90,...,0.0,NaN,0.0,0.0,0.0,0.0,0.0,74.0,74.1,0.1
1,1322,NaN,NaN,NaN,0,327,1,92,0,10,...,0.0,NaN,0.0,0.0,0.0,0.0,0.0,75.3,69.2,-6.1
2,1319,NaN,NaN,NaN,0,255,1,78,1,5,...,-1.0,NaN,0.0,0.0,0.0,0.0,0.0,46.0,35.7,-10.3
3,1333,NaN,NaN,NaN,0,293,1,82,1,10,...,0.0,NaN,0.0,0.0,0.0,0.0,0.0,58.7,53.4,-5.3
4,1452,NaN,NaN,NaN,0,341,0,80,1,10,...,-1.0,NaN,0.0,0.0,0.0,0.0,0.0,40.7,36.5,-4.2


In [32]:
ex_X = external[features].drop(columns=['死亡標記'])
ex_y = external['死亡標記']

In [33]:
ex_X = ex_X.fillna(0)

In [34]:
# 丟掉這些欄位
ex_X = ex_X.drop(columns=drop_columns)

In [35]:
ex_X.describe().T

,count,mean,std,min,25%,50%,75%,max
性別_is_male,6216.0,0.510457,0.499931,0.0,0.0000,1.0,1.000000,1.000000
預估年齡,6216.0,78.604086,11.724613,1.0,72.0000,81.0,87.000000,124.000000
DNR_flag,6216.0,0.455598,0.498065,0.0,0.0000,0.0,1.000000,1.000000
ADL_總分_max,6216.0,28.906853,31.175410,0.0,0.0000,20.0,50.000000,100.000000
ADL_明顯惡化,6216.0,0.111326,0.314560,0.0,0.0000,0.0,0.000000,1.000000
六個月內住院次數,6216.0,0.712677,1.088878,0.0,0.0000,0.0,1.000000,11.000000
ADL_last_CouldNot,6216.0,0.010618,0.102502,0.0,0.0000,0.0,0.000000,1.000000
ADL_first_CouldNot,6216.0,0.010618,0.102502,0.0,0.0000,0.0,0.000000,1.000000
ADL_first_score,6216.0,26.106821,30.025157,0.0,0.0000,15.0,45.000000,100.000000
ADL_Max,6216.0,28.906853,31.175410,0.0,0.0000,20.0,50.000000,100.000000


In [36]:
eX_missing_info = ex_X.isnull().sum().to_frame(name='Missing Count')
eX_missing_info['Missing Ratio'] = (eX_missing_info['Missing Count'] / len(ex_X)).round(4)
eX_missing_info = eX_missing_info.sort_values(by='Missing Ratio', ascending=False)
eX_missing_info

,Missing Count,Missing Ratio
性別_is_male,0,0.0
預估年齡,0,0.0
DNR_flag,0,0.0
ADL_總分_max,0,0.0
ADL_明顯惡化,0,0.0
六個月內住院次數,0,0.0
ADL_last_CouldNot,0,0.0
ADL_first_CouldNot,0,0.0
ADL_first_score,0,0.0
ADL_Max,0,0.0


In [37]:
from sklearn.impute import SimpleImputer
from sklearn.base import clone
from sklearn.exceptions import NotFittedError
from sklearn.utils.validation import check_is_fitted
import plotly.subplots as sp

def evaluate_all_models_visual(models: dict, X_val, y_val):
    mean_fpr = np.linspace(0, 1, 100)
    fig_roc = go.Figure()
    results = []

    # 建立混淆矩陣子圖
    num_models = len(models)
    cols = 3
    rows = int(np.ceil(num_models / cols))

    fig_cm = sp.make_subplots(
        rows=rows, cols=cols,
        subplot_titles=list(models.keys()),
        horizontal_spacing=0.15,
        vertical_spacing=0.15
    )

    for i, (model_name, model) in enumerate(models.items()):
        print(f"🔍 Evaluating {model_name}...")

        # 嘗試使用原始資料
        X_input = X_val.copy()
        y_input = y_val

        # 若模型不支援 NaN，則補值
        try:
            # 嘗試呼叫 predict_proba
            _ = model.predict_proba(X_input)
        except ValueError as e:
            if "Input X contains NaN" in str(e):
                print(f"⚠️  {model_name} 不支援 NaN，自動補值中...")
                imputer = SimpleImputer(strategy="median")
                X_input = pd.DataFrame(imputer.fit_transform(X_input), columns=X_val.columns)
            else:
                raise e

        # 預測
        y_pred = model.predict(X_input)
        y_prob = model.predict_proba(X_input)[:, 1]

        # 指標
        acc = accuracy_score(y_input, y_pred)
        prec = precision_score(y_input, y_pred)
        rec = recall_score(y_input, y_pred)
        f1 = f1_score(y_input, y_pred)
        auc_val = roc_auc_score(y_input, y_prob)

        # ROC
        fpr, tpr, _ = roc_curve(y_input, y_prob)
        tpr_interp = np.interp(mean_fpr, fpr, tpr)
        tpr_interp[0], tpr_interp[-1] = 0.0, 1.0

        fig_roc.add_trace(go.Scatter(
            x=mean_fpr, y=tpr_interp,
            mode='lines',
            name=f"{model_name} (AUC={auc_val:.3f})"
        ))

        # 混淆矩陣
        cm = confusion_matrix(y_input, y_pred)
        row, col = i // cols + 1, i % cols + 1
        fig_cm.add_trace(
            go.Heatmap(
                z=cm,
                x=["Predicted Negative", "Predicted Positive"],
                y=["Actual Negative", "Actual Positive"],
                colorscale='Blues',
                text=cm,
                texttemplate="%{text}",
                showscale=False
            ),
            row=row, col=col
        )

        results.append({
            "Model": model_name,
            "Accuracy": acc,
            "Precision": prec,
            "Recall": rec,
            "F1 Score": f1,
            "ROC AUC": auc_val
        })

    # 隨機基準線
    fig_roc.add_trace(go.Scatter(
        x=[0, 1], y=[0, 1],
        mode='lines',
        line=dict(dash='dash'),
        name='Random Baseline'
    ))

    fig_roc.update_layout(
        title="ROC Curve Comparison",
        xaxis_title="False Positive Rate",
        yaxis_title="True Positive Rate",
        width=800,
        height=600
    )
    fig_roc.show()

    fig_cm.update_layout(
        title="Confusion Matrices of All Models",
        width=400 * cols,
        height=300 * rows,
        showlegend=False
    )
    fig_cm.show()

    # 指標表格
    df_result = pd.DataFrame(results)

    return df_result

In [38]:
# 假設已經訓練完模型並存在 trained_models 中
evaluate_all_models_visual(trained_models, ex_X, ex_y)

🔍 Evaluating HybridXGBRF (Our Approach)...
🔍 Evaluating LogisticRegression (max_iter=200)...
🔍 Evaluating XGBClassifier...
🔍 Evaluating RandomForestClassifier...
🔍 Evaluating LogisticRegression (max_iter=1000)...
🔍 Evaluating Ridge...
🔍 Evaluating Lasso...
🔍 Evaluating Elastic...


,Model,Accuracy,Precision,Recall,F1 Score,ROC AUC
0,HybridXGBRF (Our Approach),0.831564,0.852885,0.498035,0.628855,0.887272
1,LogisticRegression (max_iter=200),0.744208,0.645802,0.237507,0.347291,0.805160
2,XGBClassifier,0.828024,0.857430,0.479506,0.615052,0.886864
3,RandomForestClassifier,0.818855,0.788039,0.503088,0.614119,0.863374
4,LogisticRegression (max_iter=1000),0.742921,0.648298,0.224593,0.333611,0.806362
5,Ridge,0.742921,0.649266,0.223470,0.332498,0.805833
6,Lasso,0.743082,0.650327,0.223470,0.332637,0.806011
7,Elastic,0.743243,0.650897,0.224031,0.333333,0.805911


In [39]:
# ===== 次族群實驗：年齡、ADL 變化、性別 =====
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def _first_existing_column(df, candidates):
    for c in candidates:
        if c in df.columns:
            return c
    return None

def _make_sex_masks(df):
    """
    回傳 {'男性': mask, '女性': mask}；若無法判斷，回傳空 dict。
    支援：
      1) '性別' 欄位（值可能為 '男'/'女' 或 1/0/2…）
      2) one-hot 欄位 '性別_男' / '性別_女'
    """
    masks = {}
    if '性別_is_male' in df.columns:
        col = df['性別_is_male']
        # 嘗試各種常見標記
        male_mask = col.astype(str).str.contains('男') | (col == 1) | (col.astype(str).str.lower().isin(['m','male']))
        female_mask = col.astype(str).str.contains('女') | (col == 0) | (col.astype(str).str.lower().isin(['f','female']))
        if male_mask.any(): masks['男性'] = male_mask
        if female_mask.any(): masks['女性'] = female_mask
    else:
        male_col = _first_existing_column(df, ['性別_男','男','male','Male','M'])
        female_col = _first_existing_column(df, ['性別_女','女','female','Female','F'])
        if male_col is not None:
            masks['男性'] = df[male_col] == 1
        if female_col is not None:
            masks['女性'] = df[female_col] == 1
    return masks

def _make_adl_change_masks(df):
    """
    建立 ADL 變好/變差遮罩：
    'ADL_明顯惡化'（=0 視為變好，=1 變差）
    """
    masks = {}
    masks['ADL 變好'] = df['ADL_明顯惡化'] == 0
    masks['ADL 變差'] = df['ADL_明顯惡化'] == 1
    return masks

def _compute_metrics(y_true, y_prob, y_pred):
    return {
        'Accuracy': accuracy_score(y_true, y_pred),
        'Precision': precision_score(y_true, y_pred, zero_division=0),
        'Recall': recall_score(y_true, y_pred, zero_division=0),
        'F1': f1_score(y_true, y_pred, zero_division=0),
        'ROC AUC': roc_auc_score(y_true, y_prob) if len(np.unique(y_true)) > 1 else np.nan,
        'Support (n)': int(len(y_true)),
        'Positives (n)': int(y_true.sum())
    }

def evaluate_subgroups(models: dict, X_all: pd.DataFrame, y_all: pd.Series, raw_df_for_masks: pd.DataFrame):
    """
    models: 已訓練好的模型字典 trained_models
    X_all, y_all: 用於評估的特徵與標記（例如 ex_X, ex_y）
    raw_df_for_masks: 與 X_all 對齊、包含「年齡/ADL/性別」原始欄位的 DataFrame（例如 external）
    """
    # 年齡遮罩（需 '預估年齡'）
    subgroup_masks = {}
    if '預估年齡' in raw_df_for_masks.columns:
        subgroup_masks['年齡 > 85'] = raw_df_for_masks['預估年齡'] > 85
        subgroup_masks['年齡 <= 85'] = raw_df_for_masks['預估年齡'] <= 85
    else:
        print("⚠️ 找不到欄位『預估年齡』，跳過年齡分組。")

    # ADL 變化遮罩
    adl_masks = _make_adl_change_masks(raw_df_for_masks)
    if adl_masks:
        subgroup_masks.update(adl_masks)
    else:
        print("⚠️ 找不到可推算 ADL 變化的欄位，跳過 ADL 分組。")

    # 性別遮罩
    sex_masks = _make_sex_masks(raw_df_for_masks)
    if sex_masks:
        subgroup_masks.update(sex_masks)
    else:
        print("⚠️ 找不到可用的性別欄位，跳過性別分組。")

    rows = []
    for subgroup_name, mask in subgroup_masks.items():
        mask = mask.fillna(False).astype(bool)  # 安全轉型
        if mask.sum() == 0:
            print(f"⚠️ 次族群「{subgroup_name}」資料筆數為 0，略過。")
            continue

        X_sub = X_all.loc[mask]
        y_sub = y_all.loc[mask]

        # 若模型不支援 NaN，與你上面一致，統一補值策略（中位數）
        from sklearn.impute import SimpleImputer
        imputer = SimpleImputer(strategy="median")
        X_sub_imp = pd.DataFrame(imputer.fit_transform(X_sub), columns=X_sub.columns, index=X_sub.index)

        for model_name, model in models.items():
            # 預測
            y_prob = model.predict_proba(X_sub_imp)[:, 1]
            y_pred = (y_prob >= 0.5).astype(int)

            metrics = _compute_metrics(y_sub, y_prob, y_pred)
            metrics.update({'Subgroup': subgroup_name, 'Model': model_name})
            rows.append(metrics)

    result_df = pd.DataFrame(rows)
    if not result_df.empty:
        # 排序：先按次族群，再按 ROC AUC 由高到低
        result_df = result_df.sort_values(by=['Subgroup','ROC AUC'], ascending=[True, False]).reset_index(drop=True)
    return result_df

# === 執行：以外部驗證集為例 ===
subgroup_results = evaluate_subgroups(trained_models, ex_X, ex_y, external)
display(subgroup_results)

,Accuracy,Precision,Recall,F1,ROC AUC,Support (n),Positives (n),Subgroup,Model
0,0.831825,0.855610,0.528951,0.653746,0.897126,5524,1658,ADL 變好,HybridXGBRF (Our Approach)
1,0.827842,0.861083,0.508444,0.639363,0.897020,5524,1658,ADL 變好,XGBClassifier
2,0.820782,0.789931,0.548854,0.647687,0.872286,5524,1658,ADL 變好,RandomForestClassifier
3,0.733526,0.658163,0.233414,0.344613,0.811814,5524,1658,ADL 變好,LogisticRegression (max_iter=1000)
4,0.733888,0.660410,0.233414,0.344920,0.811516,5524,1658,ADL 變好,Lasso
5,0.734070,0.660988,0.234017,0.345657,0.811405,5524,1658,ADL 變好,Elastic
6,0.733888,0.659864,0.234017,0.345503,0.811317,5524,1658,ADL 變好,Ridge
7,0.734794,0.654896,0.246080,0.357738,0.810558,5524,1658,ADL 變好,LogisticRegression (max_iter=200)
8,0.829480,0.666667,0.081301,0.144928,0.733450,692,123,ADL 變差,HybridXGBRF (Our Approach)
9,0.829480,0.647059,0.089431,0.157143,0.730271,692,123,ADL 變差,XGBClassifier


In [40]:
# 若只想看主力模型
display(subgroup_results[subgroup_results['Model'] == 'HybridXGBRF (Our Approach)'])

,Accuracy,Precision,Recall,F1,ROC AUC,Support (n),Positives (n),Subgroup,Model
0,0.831825,0.855610,0.528951,0.653746,0.897126,5524,1658,ADL 變好,HybridXGBRF (Our Approach)
8,0.829480,0.666667,0.081301,0.144928,0.733450,692,123,ADL 變差,HybridXGBRF (Our Approach)
17,0.854749,0.834975,0.474790,0.605357,0.890524,3043,714,女性,HybridXGBRF (Our Approach)
24,0.833411,0.837681,0.489416,0.617851,0.884112,4292,1181,年齡 <= 85,HybridXGBRF (Our Approach)
32,0.827443,0.882857,0.515000,0.650526,0.893553,1924,600,年齡 > 85,HybridXGBRF (Our Approach)
40,0.809329,0.864353,0.513590,0.644327,0.881697,3173,1067,男性,HybridXGBRF (Our Approach)


In [41]:
# ===== 複合次族群：年齡 × 性別 × ADL是否明顯惡化 =====
from itertools import product
import pandas as pd
from sklearn.impute import SimpleImputer

def evaluate_age_sex_composites(
    models: dict,
    X_all: pd.DataFrame,
    y_all: pd.Series,
    raw_df_for_masks: pd.DataFrame,
    age_threshold: int = 85,
    min_support: int = 10   # 次族群最少樣本數，太小就略過避免不穩定
):
    """
    針對「年齡(>threshold / <=threshold) × 性別(男性/女性) × ADL(變好/變差)」的交叉次族群做評估。
    會輸出每個模型在各交叉次族群的 Accuracy / Precision / Recall / F1 / ROC-AUC 等。
    依賴你已定義的: _make_sex_masks, _make_adl_change_masks, _compute_metrics。
    """
    if '預估年齡' not in raw_df_for_masks.columns:
        print("⚠️ 找不到欄位『預估年齡』，無法建立年齡遮罩。")
        return pd.DataFrame()

    # 年齡 bins
    age_masks = {
        f'年齡 > {age_threshold}': (raw_df_for_masks['預估年齡'] > age_threshold),
        f'年齡 <= {age_threshold}': (raw_df_for_masks['預估年齡'] <= age_threshold),
    }

    # 性別 masks（沿用你上面的 _make_sex_masks）
    sex_masks = _make_sex_masks(raw_df_for_masks)
    if not sex_masks:
        print("⚠️ 找不到可用的性別欄位，無法建立性別遮罩。")
        return pd.DataFrame()

    # ADL 變化 masks（沿用你上面的 _make_adl_change_masks）
    if 'ADL_明顯惡化' not in raw_df_for_masks.columns:
        print("⚠️ 找不到欄位『ADL_明顯惡化』，無法建立 ADL 遮罩。")
        return pd.DataFrame()
    adl_masks = _make_adl_change_masks(raw_df_for_masks)  # {'ADL 變好': mask, 'ADL 變差': mask}

    rows = []
    for (age_name, age_mask), (sex_name, sex_mask), (adl_name, adl_mask) in product(
        age_masks.items(), sex_masks.items(), adl_masks.items()
    ):
        combo_name = f"{age_name} & {sex_name} & {adl_name}"
        mask = (
            age_mask.fillna(False).astype(bool)
            & sex_mask.fillna(False).astype(bool)
            & adl_mask.fillna(False).astype(bool)
        )
        n = int(mask.sum())
        if n < min_support:
            print(f"ℹ️ 複合次族群「{combo_name}」樣本數 {n} < min_support={min_support}，略過。")
            continue

        X_sub = X_all.loc[mask]
        y_sub = y_all.loc[mask]

        # 與你現有策略一致：補缺失值（中位數）
        imputer = SimpleImputer(strategy="median")
        X_sub_imp = pd.DataFrame(imputer.fit_transform(X_sub), columns=X_sub.columns, index=X_sub.index)

        for model_name, model in models.items():
            y_prob = model.predict_proba(X_sub_imp)[:, 1]
            y_pred = (y_prob >= 0.5).astype(int)

            metrics = _compute_metrics(y_sub, y_prob, y_pred)
            metrics.update({'Subgroup': combo_name, 'Model': model_name})
            rows.append(metrics)

    result_df = pd.DataFrame(rows)
    if not result_df.empty:
        result_df['Prevalence'] = result_df['Positives (n)'] / result_df['Support (n)']
        result_df = result_df.sort_values(by=['Subgroup', 'ROC AUC'], ascending=[True, False]).reset_index(drop=True)
    return result_df

In [42]:
age_sex_results = evaluate_age_sex_composites(
    trained_models, ex_X, ex_y, external,
    age_threshold=85,
    min_support=10
)
display(age_sex_results)

pivot_auc = age_sex_results.pivot_table(index='Subgroup', columns='Model', values='ROC AUC')
display(pivot_auc)

,Accuracy,Precision,Recall,F1,ROC AUC,Support (n),Positives (n),Subgroup,Model,Prevalence
0,0.858369,0.860335,0.427778,0.571429,0.896758,1631,360,年齡 <= 85 & 女性 & ADL 變好,XGBClassifier,0.220723
1,0.865113,0.824074,0.494444,0.618056,0.895268,1631,360,年齡 <= 85 & 女性 & ADL 變好,HybridXGBRF (Our Approach),0.220723
2,0.855303,0.762712,0.500000,0.604027,0.873938,1631,360,年齡 <= 85 & 女性 & ADL 變好,RandomForestClassifier,0.220723
3,0.784795,0.556962,0.122222,0.200456,0.810476,1631,360,年齡 <= 85 & 女性 & ADL 變好,LogisticRegression (max_iter=200),0.220723
4,0.784181,0.558824,0.105556,0.177570,0.810164,1631,360,年齡 <= 85 & 女性 & ADL 變好,LogisticRegression (max_iter=1000),0.220723
...,...,...,...,...,...,...,...,...,...,...
59,0.782178,0.454545,0.238095,0.312500,0.667857,101,21,年齡 > 85 & 男性 & ADL 變差,LogisticRegression (max_iter=200),0.207921
60,0.792079,0.500000,0.238095,0.322581,0.655952,101,21,年齡 > 85 & 男性 & ADL 變差,LogisticRegression (max_iter=1000),0.207921
61,0.792079,0.500000,0.238095,0.322581,0.642262,101,21,年齡 > 85 & 男性 & ADL 變差,Lasso,0.207921
62,0.792079,0.500000,0.238095,0.322581,0.640476,101,21,年齡 > 85 & 男性 & ADL 變差,Ridge,0.207921


Model,Elastic,HybridXGBRF (Our Approach),Lasso,LogisticRegression (max_iter=1000),LogisticRegression (max_iter=200),RandomForestClassifier,Ridge,XGBClassifier
Subgroup,,,,,,,,
年齡 <= 85 & 女性 & ADL 變好,0.809755,0.895268,0.809886,0.810164,0.810476,0.873938,0.809687,0.896758
年齡 <= 85 & 女性 & ADL 變差,0.682163,0.734848,0.681474,0.680957,0.684056,0.734762,0.683023,0.725465
年齡 <= 85 & 男性 & ADL 變好,0.801395,0.889126,0.801544,0.801927,0.798900,0.862250,0.801322,0.888159
年齡 <= 85 & 男性 & ADL 變差,0.719931,0.754397,0.718617,0.717000,0.709319,0.729129,0.720740,0.743380
年齡 > 85 & 女性 & ADL 變好,0.832909,0.907846,0.832909,0.833061,0.832291,0.876408,0.832819,0.907567
年齡 > 85 & 女性 & ADL 變差,0.666667,0.719421,0.666200,0.661998,0.644725,0.702381,0.668534,0.716153
年齡 > 85 & 男性 & ADL 變好,0.761783,0.894867,0.761933,0.761912,0.763122,0.865626,0.761537,0.888507
年齡 > 85 & 男性 & ADL 變差,0.639881,0.695833,0.642262,0.655952,0.667857,0.671429,0.640476,0.715476


In [43]:
# 每個模型在哪個次族群表現最好（同樣以 ROC AUC 為主）
def best_subgroup_per_model(results_df: pd.DataFrame,
                            primary='ROC AUC',
                            tie_breakers=('F1','Recall','Precision','Accuracy','Support (n)')):
    sort_cols = ['Model', primary, *tie_breakers]
    sort_asc  = [True, False, *([False]*len(tie_breakers))]
    df_sorted = results_df.sort_values(by=sort_cols, ascending=sort_asc)
    best_df = df_sorted.groupby('Model', as_index=False).head(1).reset_index(drop=True)
    return best_df

best_subgroup_each_model = best_subgroup_per_model(age_sex_results)
display(best_subgroup_each_model)

,Accuracy,Precision,Recall,F1,ROC AUC,Support (n),Positives (n),Subgroup,Model,Prevalence
0,0.734522,0.638889,0.151815,0.245333,0.832909,1066,303,年齡 > 85 & 女性 & ADL 變好,Elastic,0.28424
1,0.839587,0.858696,0.521452,0.648871,0.907846,1066,303,年齡 > 85 & 女性 & ADL 變好,HybridXGBRF (Our Approach),0.28424
2,0.734522,0.638889,0.151815,0.245333,0.832909,1066,303,年齡 > 85 & 女性 & ADL 變好,Lasso,0.28424
3,0.735460,0.643836,0.155116,0.250000,0.833061,1066,303,年齡 > 85 & 女性 & ADL 變好,LogisticRegression (max_iter=1000),0.28424
4,0.740150,0.675676,0.165017,0.265252,0.832291,1066,303,年齡 > 85 & 女性 & ADL 變好,LogisticRegression (max_iter=200),0.28424
5,0.815197,0.752381,0.521452,0.615984,0.876408,1066,303,年齡 > 85 & 女性 & ADL 變好,RandomForestClassifier,0.28424
6,0.734522,0.638889,0.151815,0.245333,0.832819,1066,303,年齡 > 85 & 女性 & ADL 變好,Ridge,0.28424
7,0.829268,0.862275,0.475248,0.612766,0.907567,1066,303,年齡 > 85 & 女性 & ADL 變好,XGBClassifier,0.28424


In [44]:
# 直接列出「整體表現最高的 (Subgroup, Model) Top-K」
def top_k_overall(results_df: pd.DataFrame, k=10,
                  primary='ROC AUC',
                  tie_breakers=('F1','Recall','Precision','Accuracy','Support (n)')):
    sort_cols = [primary, *tie_breakers]
    sort_asc  = [False, *([False]*len(tie_breakers))]
    return results_df.sort_values(by=sort_cols, ascending=sort_asc).head(k).reset_index(drop=True)

top10 = top_k_overall(age_sex_results, k=10)
display(top10)

,Accuracy,Precision,Recall,F1,ROC AUC,Support (n),Positives (n),Subgroup,Model,Prevalence
0,0.839587,0.858696,0.521452,0.648871,0.907846,1066,303,年齡 > 85 & 女性 & ADL 變好,HybridXGBRF (Our Approach),0.284240
1,0.829268,0.862275,0.475248,0.612766,0.907567,1066,303,年齡 > 85 & 女性 & ADL 變好,XGBClassifier,0.284240
2,0.858369,0.860335,0.427778,0.571429,0.896758,1631,360,年齡 <= 85 & 女性 & ADL 變好,XGBClassifier,0.220723
3,0.865113,0.824074,0.494444,0.618056,0.895268,1631,360,年齡 <= 85 & 女性 & ADL 變好,HybridXGBRF (Our Approach),0.220723
4,0.800000,0.918750,0.569767,0.703349,0.894867,620,258,年齡 > 85 & 男性 & ADL 變好,HybridXGBRF (Our Approach),0.416129
5,0.812415,0.847312,0.534600,0.655574,0.889126,2207,737,年齡 <= 85 & 男性 & ADL 變好,HybridXGBRF (Our Approach),0.333937
6,0.793548,0.865169,0.596899,0.706422,0.888507,620,258,年齡 > 85 & 男性 & ADL 變好,XGBClassifier,0.416129
7,0.814227,0.859341,0.530529,0.656040,0.888159,2207,737,年齡 <= 85 & 男性 & ADL 變好,XGBClassifier,0.333937
8,0.815197,0.752381,0.521452,0.615984,0.876408,1066,303,年齡 > 85 & 女性 & ADL 變好,RandomForestClassifier,0.284240
9,0.855303,0.762712,0.500000,0.604027,0.873938,1631,360,年齡 <= 85 & 女性 & ADL 變好,RandomForestClassifier,0.220723


In [45]:
# 每個次族群下，表現最好的模型（以 ROC AUC 為主，F1/Recall/Precision/Accuracy/Support 作為平手時的次序）
def best_model_per_subgroup(results_df: pd.DataFrame,
                            primary='ROC AUC',
                            tie_breakers=('F1','Recall','Precision','Accuracy','Support (n)')):
    sort_cols = ['Subgroup', primary, *tie_breakers]
    sort_asc  = [True, False, *([False]*len(tie_breakers))]
    df_sorted = results_df.sort_values(by=sort_cols, ascending=sort_asc)
    # 取每個 Subgroup 的第一列（即最佳模型）
    best_df = df_sorted.groupby('Subgroup', as_index=False).head(1).reset_index(drop=True)
    return best_df

best_by_subgroup = best_model_per_subgroup(age_sex_results)
display(best_by_subgroup)

,Accuracy,Precision,Recall,F1,ROC AUC,Support (n),Positives (n),Subgroup,Model,Prevalence
0,0.858369,0.860335,0.427778,0.571429,0.896758,1631,360,年齡 <= 85 & 女性 & ADL 變好,XGBClassifier,0.220723
1,0.842105,0.500000,0.060606,0.108108,0.734848,209,33,年齡 <= 85 & 女性 & ADL 變差,HybridXGBRF (Our Approach),0.157895
2,0.812415,0.847312,0.534600,0.655574,0.889126,2207,737,年齡 <= 85 & 男性 & ADL 變好,HybridXGBRF (Our Approach),0.333937
3,0.804082,0.800000,0.078431,0.142857,0.754397,245,51,年齡 <= 85 & 男性 & ADL 變差,HybridXGBRF (Our Approach),0.208163
4,0.839587,0.858696,0.521452,0.648871,0.907846,1066,303,年齡 > 85 & 女性 & ADL 變好,HybridXGBRF (Our Approach),0.284240
5,0.868613,0.500000,0.055556,0.100000,0.719421,137,18,年齡 > 85 & 女性 & ADL 變差,HybridXGBRF (Our Approach),0.131387
6,0.800000,0.918750,0.569767,0.703349,0.894867,620,258,年齡 > 85 & 男性 & ADL 變好,HybridXGBRF (Our Approach),0.416129
7,0.801980,0.600000,0.142857,0.230769,0.715476,101,21,年齡 > 85 & 男性 & ADL 變差,XGBClassifier,0.207921
